In [1]:
%%capture
!pip install -r requirements.txt

# Tubemaps

In [4]:
london_stations = pd.read_csv("../dataset/tubemaps/london.stations.csv")
london_connections = pd.read_csv("../dataset/tubemaps/london.connections.csv")
london_lines = pd.read_csv("../dataset/tubemaps/london.lines.csv")

In [8]:
# Map Bounds to limit stations of interest
lat_max = 51.5345
lat_min = 51.4860
long_max = -0.0725
long_min = -0.1527

london_stations_filtered = london_stations[(london_stations['latitude'] <= lat_max) & 
                                        (london_stations['latitude'] >= lat_min) & 
                                        (london_stations['longitude'] <= long_max) & 
                                        (london_stations['longitude'] >= long_min)]
london_stations_filtered = london_stations_filtered.drop(columns=['Unnamed: 0']).drop_duplicates()
london_stations_filtered.to_csv("dataset/stations.csv", index=False)

In [10]:
london_connections_filtered =  london_connections[(london_connections['station1'].isin(london_stations_filtered['id'])) & (london_connections['station2'].isin(london_stations_filtered['id']))]
london_connections_filtered = london_connections_filtered.drop_duplicates(subset=['station1','station2'])
london_connections_filtered.to_csv("dataset/connections.csv", index=False)

,station1,station2,line,time
2,49,87,1,1
3,49,197,1,2
6,84,148,1,3
7,87,279,1,2
16,148,279,1,1
...,...,...,...,...
398,107,273,11,2
400,192,277,11,2
401,198,272,11,1
402,198,273,11,3


In [68]:
london_lines.to_csv("dataset/lines.csv", index=False)

# DB Creation

In [5]:
import pandas as pd
london_stations = pd.read_csv("dataset/stations.csv")
london_connections = pd.read_csv("dataset/connections.csv")
london_lines = pd.read_csv("dataset/lines.csv")

facts = ""

def remove_spl_chars(text):
    return text.lower().replace(" ","_").replace("'","").replace("(","_").replace(")","_").replace("&","and").replace(".","").replace(",","")

# ETA
for station1, station2, line, time in london_connections.values.tolist():  
        station1_name = remove_spl_chars(london_stations[london_stations['id']==station1]['name'].values[0])
        station2_name = remove_spl_chars(london_stations[london_stations['id']==station2]['name'].values[0])
        facts += f"eta({station1_name},{station2_name},{time}).\n"
        facts += f"eta({station2_name},{station1_name},{time}).\n"
# facts += "eta(Node, NextNode, Time):- !,eta(NextNode, Node, Time).\n"
for id, lat, long, name, display_name, zone, tot_line, rail, prolog_name in london_stations.values.tolist():  
        facts += f"location({prolog_name},{lat},{long}).\n"

text_file = open("src/facts.pl", "w")
n = text_file.write(facts)
text_file.close()


In [6]:
from swiplserver import PrologMQI

with PrologMQI() as mqi:
    with mqi.create_thread() as prolog_thread:
        result = prolog_thread.query("consult(\"./src/facts.pl\").")
        print(result)
        result = prolog_thread.query("consult(\"./src/search_algorithms.pl\").")
        print(result)
        origin = remove_spl_chars("Temple")
        destination = remove_spl_chars("Cannon Street")        
        print(f"Path: {origin} - - to - - {destination}")
        query = f"breadth_first_search({origin}, {destination}, Path)."
        result = prolog_thread.query_async(query, find_all=False)
        result = prolog_thread.query_async_result()
        print(result)

True
True
Path: temple - - to - - cannon_street
[{'Path': ['temple', 'blackfriars', 'mansion_house', 'cannon_street']}]
